<a href="https://colab.research.google.com/github/lauroPereira/data-wrangling-lessons/blob/master/data_wrangling_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px

from sklearn import datasets
from sklearn.preprocessing import scale, minmax_scale, power_transform

from IPython.core.display import HTML
from ipywidgets import interact, widgets

In [ ]:
sns.set_theme(
    context='talk',
    style='ticks',
    font_scale=.8,
    rc={
        'figure.figsize': (12,8)
    }
)

In [ ]:
def display_side_by_side(dataframes:list,titles:list):
  html_str=''
  for df,title in zip(dataframes, titles):
    html_str+=f'<span>{title}:</span>'
    html_str+=df.to_html().replace('table','table style="display:inline"')
    html_str+='&nbsp'*10
  display(HTML(html_str))

# Explicação: método merge

In [ ]:
df1 = pd.DataFrame(
    {
        "key": ["K0","K1","K2","K3"],
        "A": ["A0","A1","A2","A3"],
        "B": ["B0","B1","B2","B3"]
    }, index=range(4)
)

df2 = pd.DataFrame(
    {
        "key": ["K1","K2","K3","K4"],
        "C": ["C0","C1","C2","C3"],
        "D": ["D0","D1","D2","D3"]
    }, index=range(4)
)

In [ ]:
display_side_by_side([df1, df2], ['df1', 'df2'])

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3
,key,C,D
0,K1,C0,D0
1,K2,C1,D1
2,K3,C2,D2
3,K4,C3,D3


In [ ]:
@interact(method=['inner','left','right','outer'])
def merge(method):
  df_merge = df1.merge(df2, how=method)
  display_side_by_side(
      [df1, df2, df_merge],
      ['df1', 'df2', f'{method}_join']
  )

interactive(children=(Dropdown(description='method', options=('inner', 'left', 'right', 'outer'), value='inner…

# Explicação: método Concat

In [ ]:
df_A = df1.iloc[:2]
df_B = df1.iloc[-2:]

display_side_by_side([df_A,df_B], ['df_A','df_B'])

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
,key,A,B
2,K2,A2,B2
3,K3,A3,B3


In [ ]:
pd.concat([df_A,df_B])

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,B3


In [ ]:
df_test1 = pd.concat([df_A,df_B], axis=1)
df_test2 = pd.concat([df_A,df_B.reset_index(drop=True)], axis=1)
display_side_by_side([df_test1,df_test2], ['concat (mantendo index)','concat (forçando index)'])

,key,A,B,key,A,B
0,K0,A0,B0,NaN,NaN,NaN
1,K1,A1,B1,NaN,NaN,NaN
2,NaN,NaN,NaN,K2,A2,B2
3,NaN,NaN,NaN,K3,A3,B3
,key,A,B,key,A,B
0,K0,A0,B0,K2,A2,B2
1,K1,A1,B1,K3,A3,B3


# Explicação: método JOIN

***A diferença para o merge é que o JOIN faz a ligação apenas pelo index do dataframe***

In [ ]:
df_left = pd.DataFrame(
    {
        "A": ["A0","A1","A2","A3"],
        "B": ["B0","B1","B2","B3"]
    }, index=["K0","K1","K2","K3"]
)

df_right = pd.DataFrame(
    {
        "C": ["C0","C1","C2","C3"],
        "D": ["D0","D1","D2","D3"]
    }, index=["K0","K2","K4","K6"]
)

In [ ]:
@interact(method=['inner','left','right','outer'])
def join(method):
  df_join = df_left.join(df_right, how=method)
  display_side_by_side([df_left, df_right, df_join], ['df_left', 'df_right', f'df_join_{method}'])

interactive(children=(Dropdown(description='method', options=('inner', 'left', 'right', 'outer'), value='inner…

#Explicação: método GROUP BY 

## Funcionamento básico
***Executa 3 etapas no dataframe:***
* ***SPLIT: Separa em subsdataframes para cada categoria informada.***
* ***APPLY: Aplica uma formula específicada, por exemplo: soma, média, etc***
* ***COMBINE: Une novamente os subsdataframes em um único dataset com as categorias como indice***

In [ ]:
df = sns.load_dataset('tips')
df.head()

NameError: ignored

In [ ]:
# aplicar a função a todas as colunas
df.groupby('sex').mean()

# aplicar a função somente em colunas alvo
df.groupby('sex')['tip'].mean()

# aplicar a função somente em colunas alvo retornando um dataframe
df.groupby('sex')[['tip']].mean()

# Agrupar por múltiplas colunas
df.groupby(['time','day'])[['total_bill', 'tip']].mean()

# Aplicar múltiplas funções
df.groupby('smoker')[['tip']].agg(['min', 'max', 'mean', 'std'])

# Aplicar funções customizadas simples
df.groupby('smoker')[['tip']].agg(['min', 'max', 'mean', 'std', lambda x : x.mean()+20])

# Aplicar funções customizadas complexas
def mean_diff(subdf):
  return np.mean(subdf['tip'] / subdf['total_bill'])

df.groupby('sex').apply(mean_diff)

## Transformação

In [ ]:
g = sns.boxplot(data=df,x='tip', y='sex')
sns.set(rc={'figure.figsize':(10,3)})
g.set_xticks(list(range(0,11)));

In [ ]:
# padronizando por grupo
df1 = df.groupby('sex')[['tip']].transform(scale)
df1.rename(columns={'tip': 'transf'}, inplace=True)
df1 = df.join(df1)

g = sns.boxplot(data=df1,x='transf', y='sex')
sns.set(rc={'figure.figsize':(10,3)})
g.set_xticks(list(range(-2,6)));

In [ ]:
# padronizando por grupo
df2 = df.groupby('sex').tip.rolling(3).mean().to_frame()
df2

In [ ]:
# Exemplo das mesmas informações com groupby e plot

# Groupby
print(df.groupby(['sex','smoker']).tip.sum())

# Seaborn barplot
sns.barplot(data=df, x='sex', y='tip', hue='smoker', estimator=np.sum, errorbar=('ci', False))
plt.show()

# Plotly barplot
px.bar(data_frame=df, x='sex', y='tip', color='smoker', barmode='group')

# Explicação: método Pivot

## funcionamento básico

In [ ]:
# por padrão ele contrói os valores com as médias
df.pivot_table(index='sex', columns='day', values='tip')

In [ ]:
# podem ser incluídas listas de indices, colunas ou valores
df.pivot_table(index='sex', columns='time', values=['total_bill','tip'])

In [ ]:
# pode ser alterado o valor de média para outra função (inclusive customizada)
df.pivot_table(index='sex', columns='time', values=['total_bill','tip'], aggfunc=np.median)

In [ ]:
# pode ser incluído diversos valores para cada combinação (indice x coluna) passando uma lista de funções
df.pivot_table(index='sex', columns='time', values=['total_bill','tip'], aggfunc=['min', 'max', 'mean', 'std', lambda x : x.mean()+20])

In [ ]:
# pode ser incluído diversos valores para combinações específicas passando um dicionário
agg_dict = {'tip': 'mean', 'total_bill': [np.median, 'min']}

df.pivot_table(index='sex', columns='time', aggfunc=agg_dict)

## Tabelas LONG vs WIDE

In [ ]:
# selecionando as colunas day, time e tip, a tabela pode ser considerada WIDE pois 
df_pivot = df.pivot_table(index='day', columns='time', values='tip').reset_index()
df_pivot

In [ ]:
df_pivot.melt(id_vars='day')

In [ ]:
# transformando tabela Wide em LONG para facilitar a plotagem
df_pivot.melt(id_vars='day') \
.plot.bar(backend='plotly', x='day', y='value', color='time', barmode='group')